In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
!pip install --upgrade mplfinance
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
import mplfinance as mpl 
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")


def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols


def download_data(symbols, periodd, intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}", end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,
                           interval=intervall, progress=False, show_errors=False)
        if data.empty:
           pass
        else:
            data.to_csv(f"/content/data/{symbol}.csv")
    except:
       print("Error!")
  print(" ")


def extract_data(pieces, min_range, max_range, how_many_future_candles, how_many_past_candles, each_row_past, interval_for_that):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 0
  for file in files:
      each_file_proc(file, now, index, min_range, max_range, how_many_future_candles,
                     how_many_past_candles, each_row_past, interval_for_that)

      index += 1
      print(f"File Number {index} : ", end=" ")
  print(" ")
  return now


def each_file_proc(file, now, index, min_range, max_range, how_many_future_candles, how_many_past_candles, each_row_past, interval_for_that):
    data = []
    unattached_dfs = []

    address = f"/content/data/{file}"
    datas_of_the_csv = pd.read_csv(address)
    dfs, suggs = process(datas_of_the_csv, min_range, max_range, how_many_future_candles,
                           how_many_past_candles, each_row_past, interval_for_that)

    index = 0
    all_n = np.array(dfs).shape[0]
    for df in dfs:
        date_list = []
        for i in range(np.array(df).shape[0]):
               date_list.append(datetime.fromordinal(
                   datetime.today().toordinal()+i).strftime('%Y-%m-%d'))
        df.index = date_list
        df.index.name = "Date"
        df.index = pd.to_datetime(df.index)
        right_now = datetime.now().strftime("%H%M%S%f")
        address = f"/content/extracted/{now}/{right_now}_{suggs[index]}.png"
        fig1, e = mpl.plot(df, type="candle", style="yahoo",axisoff=True,scale_padding=0.5, returnfig=True,savefig=address,tight_layout=True)  
        index += 1
        print(f"{index}/{all_n}",end=" ")
        if index % 30 == 0:
          print(" ")
    print("")

def process(data, min_range, max_range, how_many_future_candles, how_many_past_candles,each_row_past,interval_for_that):
    data = data.dropna()
    datas = []
    suggs = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    data = data.astype(float)
    for i in range(each_row_past, data.shape[0]-how_many_future_candles):
        rows = data[i-each_row_past:i, :4]
        sugg = None
        past_candles = []
        for z in range(1,6):
          past_candles.append(data[i-z][3])
        past_candles = sum(past_candles)/5
        next_candles = []
        for z in range(0,5):
          next_candles.append(data[i+z][3])
        next_candles = sum(next_candles)/5
        if next_candles > past_candles:
          suggs.append(1)
        else:
          suggs.append(0)
        x = pd.DataFrame(rows, columns=["Open", "High", "Low", "Close"])
        datas.append(x)

    return datas, np.array(suggs)
def start(min_range, max_range, how_many_future_candles, how_many_past_candles, each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet):
     data = extract_data(into_how_many_csv, min_range, max_range, how_many_future_candles, how_many_past_candles,each_row_past,interval_for_that)
     return data


In [3]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
symbols = ["btc-usd","eth-usd","trx-usd","ada-usd","bnb-usd","ltc-usd","xrp-usd"]
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 


In [6]:

min_range,max_range = -1,1
how_many_future_candles = 5
how_many_past_candles = 5
each_row_past,interval_for_that = 60,-1

into_how_many_csv = 1
csv_to_parquet = 1
folder_name = start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet)


Files In Data : 7
Processing File:
1/180 2/180 3/180 4/180 5/180 6/180 7/180 8/180 9/180 10/180 11/180 12/180 13/180 14/180 15/180 16/180 17/180 18/180 19/180 20/180 21/180 22/180 23/180 24/180 25/180 26/180 27/180 28/180 29/180 30/180  
31/180 32/180 33/180 34/180 35/180 36/180 37/180 38/180 39/180 40/180 41/180 42/180 43/180 44/180 45/180 46/180 47/180 48/180 49/180 50/180 51/180 52/180 53/180 54/180 55/180 56/180 57/180 58/180 59/180 60/180  
61/180 62/180 63/180 64/180 65/180 66/180 67/180 68/180 69/180 70/180 71/180 72/180 73/180 74/180 75/180 76/180 77/180 78/180 79/180 80/180 81/180 82/180 83/180 84/180 85/180 86/180 87/180 88/180 89/180 90/180  
91/180 92/180 93/180 94/180 95/180 96/180 97/180 98/180 99/180 100/180 101/180 102/180 103/180 104/180 105/180 106/180 107/180 108/180 109/180 110/180 111/180 112/180 113/180 114/180 115/180 116/180 117/180 118/180 119/180 120/180  
121/180 122/180 123/180 124/180 125/180 126/180 127/180 128/180 129/180 130/180 131/180 132/180 133/180 1

In [7]:
#folder_name = 132512
len(os.listdir(f"/content/extracted/{folder_name}"))

1588

In [8]:
from PIL import Image
import cv2
label = []
data  = []
files = os.listdir(f"/content/extracted/{folder_name}/")
for i, image_name in enumerate(files):
  if image_name.split(".")[1] == "png":
    image = cv2.imread(f"/content/extracted/{folder_name}"+"/"+image_name,0)
    scale_percent = 50 
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
    data.append(np.array(resized))
    sugg = image_name.split("_")[1].split(".")[0]
    label.append(int(sugg))
np.array(data).shape

(1588, 277, 383)

In [9]:
def scaler(array):
  scaler = MinMaxScaler()
  scaler.fit(array)
  return scaler.transform(array)
ddata = np.array(data)
new_dd = []
for i in ddata:
  new_data = []
  for x in i:
    nx = scaler(np.array(x).reshape(-1,1))
    new_data.append(nx)
  new_dd.append(new_data)
np.array(new_dd).shape
data = np.array(new_dd).reshape(-1,277,383)
ddata = []
new_data = []

In [11]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
x_train , x_test , y_train , y_test = train_test_split(data,label,test_size=0.1)

print(f"********** {len(x_train)} {len(x_test)}")
nytrain = []
for i in y_train:
  if i == 1:
    nytrain.append([1,0])
  else:
    nytrain.append([0,1])
nytest = []
for i in y_test:
  if i == 1:
    nytest.append([1,0])
  else:
    nytest.append([0,1])

y_train = nytrain
y_test = nytest
x_train = np.array(x_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
y_train = np.array(y_train)
data = []
label = []

********** 1429 159


In [14]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Activation,Dropout,Flatten,Dense


model = Sequential()
model.add(Conv2D(32, (3, 3),activation="relu", input_shape=(277,383,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3),activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(2,activation="sigmoid"))

model.compile(loss='categorical_crossentropy',
              optimizer='adamax',
              metrics=['accuracy'])



model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 275, 381, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 137, 190, 32)     0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 135, 188, 32)      9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 67, 94, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 65, 92, 32)        9248      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 32, 46, 32)      

In [16]:
model.fit(x_train,y_train,batch_size=32,epochs=10,validation_data=(x_test,y_test))


Epoch 1/15
45/45 [==============================] - 167s 4s/step - loss: 1.7874e-04 - accuracy: 1.0000 - val_loss: 0.6813 - val_accuracy: 0.8742
Epoch 2/15
45/45 [==============================] - 165s 4s/step - loss: 1.5467e-04 - accuracy: 1.0000 - val_loss: 0.7001 - val_accuracy: 0.8805
Epoch 3/15
45/45 [==============================] - 166s 4s/step - loss: 1.3232e-04 - accuracy: 1.0000 - val_loss: 0.7173 - val_accuracy: 0.8805
Epoch 4/15
45/45 [==============================] - 166s 4s/step - loss: 1.1370e-04 - accuracy: 1.0000 - val_loss: 0.7344 - val_accuracy: 0.8742
Epoch 5/15
45/45 [==============================] - 165s 4s/step - loss: 9.9302e-05 - accuracy: 1.0000 - val_loss: 0.7494 - val_accuracy: 0.8742
Epoch 6/15
45/45 [==============================] - 166s 4s/step - loss: 8.6329e-05 - accuracy: 1.0000 - val_loss: 0.7645 - val_accuracy: 0.8679
Epoch 7/15
45/45 [==============================] - 165s 4s/step - loss: 7.6622e-05 - accuracy: 1.0000 - val_loss: 0.7790 - val_ac